In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from splinter.browser import Browser
import time

In [2]:
browser = Browser('chrome', headless=True)


In [2]:
# Scrape the NASA Mars News Site and collect the latest News Title and Paragragh Text


url = "https://mars.nasa.gov/news/"
response = requests.get(url)

time.sleep(1)

In [5]:
#results = soup.find('h',class_='features')
soup = BeautifulSoup(response.text,'html.parser')
result = soup.find_all('div',class_='content_title')
news_title= result[0].find('a').text
result_p = soup.find_all('div',class_='image_and_description_container')
news_p = result_p[0].find('div',class_='rollover_description_inner').text
print(response.prettify())
print(news_p)


AttributeError: 'Response' object has no attribute 'prettify'

In [38]:
url = "https://twitter.com/marswxreport?lang=en"
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')


In [47]:
result = soup.find_all('div',class_='js-tweet-text-container')[0]


In [48]:
mars_weather = result.find('p').contents[0]


In [49]:
url = "https://space-facts.com/mars/"
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')


In [50]:
# Use Pandas to scrape the table containing facts about the planet including Diameter, Mass
table = soup.find_all('table')
df = pd.read_html(str(table))[0]

df = df.rename(columns={0:'',1:'value'})
df.set_index('', inplace=True)
mars_fact_html = df.to_html()


In [51]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'



browser.visit(url)
hemisphere_image_urls = []

html = browser.html
soup = BeautifulSoup(html,'html.parser')

results = soup.find_all('div', class_='item')

for result in results:
        hemisphere = {}
        newurl = 'https://astrogeology.usgs.gov'+ result.find('a')['href']
        browser.visit(newurl)
        new_html = browser.html
        soup = BeautifulSoup(new_html,'html.parser')
        sample = soup.find('div', class_='downloads')
        hemisphere['img_url'] = sample.find('a')['href']
        title = soup.find('h2',class_="title").text
        hemisphere['title'] = title.replace(' Enhanced','')
        hemisphere_image_urls.append(hemisphere)
      
        time.sleep(1)


In [52]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image 

#executable_path = {'executable_path':'/Users/jingc/chromedriver'}

url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

#browser = Browser('chrome', **executable_path)
#browser = Browser('chrome', headless=True)

browser.visit(url)
search_bar_xpath ='//*[@id="full_image"]'
search_button = browser.find_by_xpath(search_bar_xpath)[0]
search_button.click()

time.sleep(2)
moreinfo_button = browser.find_link_by_partial_text('more info')
moreinfo_button.click()
time.sleep(2)
response = browser.html
featuredimg_soup = BeautifulSoup(response, 'html.parser')
featured_img_url = 'https://www.jpl.nasa.gov'+featuredimg_soup.find('figure', class_='lede').a['href']

browser.quit()

In [53]:
mars_dict = {'news_title':news_title,
             'news_paragraph':news_p,
             'current_weather':mars_weather,
             'feartured_img':featured_img_url,
             'mars_fact':mars_fact_html,
             'hemisphere_images':hemisphere_image_urls
            }
print(mars_dict)

{'news_title': "NASA's First Deep-Space CubeSats Say: ‘Polo!’", 'news_paragraph': "MarCO is a pair of tiny spacecraft that launched with NASA's InSight lander today.", 'current_weather': 'Sol 2039 (May 02, 2018), Sunny, high 0C/32F, low -74C/-101F, pressure at 7.28 hPa, daylight 05:23-17:20', 'feartured_img': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA15253_hires.jpg', 'mars_fact': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td